## Vector Search on Azure Cache for Redis

### Set environment variables

In [1]:
import openai

from typing import List, Iterator
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
from ast import literal_eval

# Redis client library for Python
import redis

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Loading the env from the .env file
load_dotenv()

True

#### Setup Redis connection

In [2]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)
import os

REDIS_HOST =  os.environ.get("REDIS_HOST")
REDIS_PORT = os.environ.get("REDIS_PORT")
REDIS_PASSWORD = os.environ.get("REDIS_PASSWORD")
# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    ssl=True,
    password=REDIS_PASSWORD
)
# should return True
redis_client.ping()

True

#### Setup OpenAI to create embeddings for the user query

In [3]:
#setup azure open ai
openai.api_type = "azure"
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_BASE_URL")
openai.api_version = "2023-06-01-preview"

# This is the deployment name, running the ada model
EMBEDDING_MODEL = "ada-deployment"

def create_embedding(user_query):
    return openai.Embedding.create(input=user_query,engine=EMBEDDING_MODEL)['data'][0]['embedding']
    

#### Redis Vector Search

In [4]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str,
    vector_field: str, 
    return_fields: list = ["title", "category", "content", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = create_embedding(user_query=user_query)

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    return results

#### Searching of text information (product)

In [6]:
# product search
PRODUCT_TEXT_INDEX_NAME="product-text-index"
PRODUCT_SEARCH_FIELD = "content_vector"
PRODUCT_RETURN_FIELDS = ["title", "category", "content", "vector_score"]
results = search_redis(redis_client, 'show some database related products', PRODUCT_TEXT_INDEX_NAME, PRODUCT_SEARCH_FIELD, PRODUCT_RETURN_FIELDS, k=10)
for i, article in enumerate(results.docs):
        score = 1 - float(article.vector_score)
        print(f"{i}. {article.title} (Score: {round(score ,3) })")

0. Azure Database for MySQL (Score: 0.768)
1. Azure SQL Database (Score: 0.763)
2. Azure Database for MariaDB (Score: 0.763)
3. Azure Database for PostgreSQL (Score: 0.76)
4. Azure Data Catalog (Score: 0.754)
5. Azure SQL Data Warehouse (Score: 0.75)
6. Azure Cosmos DB (Score: 0.747)
7. Azure Cosmos DB (Score: 0.744)
8. Azure Database Migration Service (Score: 0.744)
9. Azure Cognitive Search (Score: 0.742)


#### Searching of doc information (employee handbook)

In [7]:
# employee search
EMP_TEXT_INDEX_NAME="employee-text-index"
EMP_SEARCH_FIELD = "chunk_content_vector"
EMP_RETURN_FIELDS = ["chunk_content"]
results = search_redis(redis_client, 'what are the company values', EMP_TEXT_INDEX_NAME, EMP_SEARCH_FIELD, EMP_RETURN_FIELDS, k=1)
for i, article in enumerate(results.docs):
        print(f"{i}. {article.chunk_content}")

0. customers.  
 
Company Values:  
1. Quality: We strive to provide the highest quality products and services to our customers.  
2. Integrity: We value honesty, respect, and t rustworthiness in all our interactions.  
3. Innovation: We encourage creativity and support new ideas and approaches to our 
business.  
4. Teamwork: We believe that by working together, we can achieve greater success.  
5. Respect: We treat all our employees, c ustomers, and partners with respect and dignity.  
6. Excellence: We strive to exceed expectations and provide excellent service.   7. Accountability: We take responsibility for our actions and hold ourselves and others 
accountable for their performance.  
8. Co mmunity: We are committed to making a positive impact in the communities in which 
we work and live.  
Performance Reviews  
 
Performance Reviews at Contoso Electronics  
 
At Contoso Electronics, we strive to ensure our employees are getting the feedback they


#### Image search

In [ ]:
## TODO

#### Hybrid search

In [ ]:
## TODO

#### Image search example

In [ ]:
## TODO